# 13. Loading and Preprocessing Data with TensorFlow

Introducing the **Data API**, a useful tool in ingesting and preprocessing large datasets. 

### The Data API

The whole API revolves around the concept of **dataset**. Here a simple example:

In [29]:
import tensorflow as tf

X = tf.range(10) # any data tensor

SyntaxError: invalid syntax (<ipython-input-29-63c05173c796>, line 1)

In [18]:
dataset = tf.data.Dataset.from_tensor_slices(X)

In [19]:
dataset

<DatasetV1Adapter shapes: (), types: tf.int32>

#### Chaining Transformations

We can apply all sorts of transformation to our datasets, and these will create new datasets. We can also chain them:

In [20]:
dataset = dataset.repeat(3).batch(7)

In [22]:
for item in dataset:
    print(item)

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.

In [30]:
print(tf.__version__)

1.15.0
